In [ ]:
import pandas as pd
%pylab inline
import matplotlib.gridspec as gridspec
from astropy.io import ascii

In [ ]:
badpixphotometry = 'NBadpix_all_exps.csv'
photometryfile = 'APP_phot_all_exps.csv'

N_CR_app_df = pd.read_csv(badpixphotometry)
N_CR_app_df = N_CR_app_df.set_index(['ID', 'Filter', 'T_Start', 'Exp_Length', 'DrizzleType'])
N_CR_app_df['FracBadPix'] = N_CR_app_df['FLUX'] / N_CR_app_df['AREA']
read_in = pd.read_csv(photometryfile)
phot_df = read_in.copy().set_index(['ID', 'Filter', 'T_Start', 'Exp_Length', 'DrizzleType'])
# Join the photometry with the fraction of bad pixels
phot_df_all_stars = phot_df.join(N_CR_app_df.FracBadPix, how='inner')

In [ ]:
badpixphotometry = 'NBadpix_all_exps_pmsstars.csv'
photometryfile = 'APP_phot_all_exps_pmsstars.csv'

N_CR_app_df = pd.read_csv(badpixphotometry)
N_CR_app_df = N_CR_app_df.set_index(['ID', 'Filter', 'T_Start', 'Exp_Length', 'DrizzleType'])
N_CR_app_df['FracBadPix'] = N_CR_app_df['FLUX'] / N_CR_app_df['AREA']
read_in = pd.read_csv(photometryfile)
phot_df = read_in.copy().set_index(['ID', 'Filter', 'T_Start', 'Exp_Length', 'DrizzleType'])
# Join the photometry with the fraction of bad pixels
phot_df_pms_stars = phot_df.join(N_CR_app_df.FracBadPix, how='inner')

In [ ]:
# Red giants
red_giant_coords = ascii.read('../HST_Guido/redgiants_coords.txt')
phot_df_redgiants = phot_df_all_stars.loc[red_giant_coords['ID']]

In [ ]:
def prepare_df(phot_df):
    # We select only the magnitudes from the single drizzled frames
    phot_df = phot_df[phot_df.index.get_level_values(4)=='SingleDrizzle']
    # Only keep apertures with max 15% flagged pixels
    phot_df = phot_df[phot_df.FracBadPix < 0.12]
    
    # Update the flux according to the number of bad pixels
    phot_df['FluxCorr'] = phot_df['FLUX'] / (1-phot_df['FracBadPix'])
    phot_df[phot_df['FluxCorr']!=phot_df['FLUX']]['CERROR'] = 'FixedBadPix'
    # Update the magnitudes for the stars that did not have one
    zmag  = {'F336W':23.46,'F438W':24.98,'F555W': 25.81, 'F814W': 24.67, 'F656N': 19.92}#[hst_filter]-0.1
    phot_df_no_MAG = phot_df[phot_df.CERROR=='BadPixels']
    phot_df['MAG'][phot_df.CERROR=='BadPixels'] = -2.5*np.log10(phot_df_no_MAG['FluxCorr']) + np.array([zmag[w] for w in phot_df_no_MAG.index.get_level_values(1).values])-0.1 +2.5*np.log10(phot_df_no_MAG['ITIME'])
    
    # Only select stars with V band magnitudes between 15 and 20
    F555W_mags = phot_df['MAG']
    selection  = (F555W_mags < 24) * (F555W_mags > 15)
    phot_df    = phot_df[selection]\
    
    # Take only values with no large shift
    phot_df    = phot_df[phot_df.XSHIFT**2+phot_df.YSHIFT**2 < 4]
    
    # Normalize observation time 
    phot_df.loc[:,'Time'] = pd.to_numeric(phot_df.index.get_level_values(2))
    phot_df.loc[:,'Time'] = phot_df['Time'] - phot_df['Time'].min()
    
    # Select only stars with limited std in magnitudes
    groupby    = phot_df.groupby(['ID', 'Filter', 'Exp_Length', 'DrizzleType'])['MAG'].std()
    groupby    = phot_df.groupby(['ID', 'Filter', 'Exp_Length', 'DrizzleType'])['MAG'].std()
    groupby    = groupby.groupby('ID').median()<1.2#groupby < 1.5
    good_stars = groupby.index[groupby]
    phot_df    = phot_df[[w in good_stars for w in phot_df.index.get_level_values(0)]]
    #phot_df    = phot_df[[w in good_stars for w in phot_df.index.get_level_values(1)]]
    #phot_df    = phot_df.droplevel([2,4]).loc[good_stars]
    
    # Select only stars with limited mag error
    phot_df   = phot_df[phot_df['MERR']<0.15]
    
    # Select some stars with enough remaining measurements
    groupby    = phot_df.reset_index(level=1).groupby('ID')['Filter'].nunique()>=3
    good_stars = groupby.index[groupby]
    phot_df    = phot_df[[w in good_stars for w in phot_df.index.get_level_values(0)]]

    groupby    = (phot_df.groupby('ID')['MAG'].nunique()>60)
    good_stars = groupby.index[groupby]
    phot_df    = phot_df[[w in good_stars for w in phot_df.index.get_level_values(0)]]
    
    
    # Select some stars with enough remaining measurements in specifically Ha 
    groupby = phot_df.groupby(['ID', 'Filter'])['MAG'].nunique()
    groupby = groupby.reset_index()
    groupby = groupby[groupby.Filter=='F656N']
    good_stars = groupby[groupby['MAG']>=7].ID.values
    phot_df    = phot_df[[w in good_stars for w in phot_df.index.get_level_values(0)]]
    return phot_df

In [ ]:
def generate_plots(phot_df, high_var=False):
    # Select those with std(Ha)> std(broadband)
    star_filter_std = phot_df.groupby(['ID', 'Filter'])['MAG'].std()
    Ha_std   = star_filter_std[star_filter_std.index.get_level_values(1)=='F656N']
    mean_std = star_filter_std.groupby('ID').mean()
    high_Ha_var_stars = mean_std.index[Ha_std.droplevel(1) >= 1*mean_std].values
    stellar_indices = phot_df.index.get_level_values(0).unique().values
    high_Ha_var_stars
    
    
    plt.figure(figsize=(18,16))
    gs1 = gridspec.GridSpec(3,3)
    gs1.update(wspace=0, hspace=0) # set the spacing between axes. 
    Nstars_plot = 9
    filterlist = phot_df.index.get_level_values(1).unique()[1:] #skip F336W
    if high_var:
        toplot = np.random.choice(high_Ha_var_stars, Nstars_plot, replace=False)
    else:
        toplot = np.random.choice(stellar_indices, Nstars_plot, replace=False)
    for nstar, starID in enumerate(toplot):
        for filter_iter, filter_ in enumerate(filterlist):
            plt.subplot(gs1[nstar])
            data = phot_df.loc[starID, filter_]
            data = data[['MAG', 'MERR', 'Time']].sort_values(by='Time')
            mags = data['MAG']
            mags = mags - np.median(mags) + filter_iter
            merr = data['MERR']
            time = data['Time']
            plt.plot(time,mags, linewidth=1.2, label=filter_)
            plt.scatter(time,mags, c='red', s=5, label='')
            F555W_mag = str(np.round(phot_df.loc[starID, 'F555W']['MAG'].median(), 1))
            plt.errorbar(time, mags, yerr=merr, c='black', ls='none', label='')
            plt.title('Star '+str(int(starID))+r'. $M_{G}=$'+F555W_mag, pad=-25, size=20)
            plt.xlim(-1,8.3)

            #plt.ylim(15.,18.5)
            if nstar not in [0,3,6]:
                plt.yticks([])
            else:
                plt.ylabel('Shifted magnitude', size=16)
                plt.yticks(size=14)
            if nstar not in [6,7,8]:
                plt.xticks([])
            else:
                plt.xticks(size=16)
                plt.xlabel(r'$\Delta \, T$ [days]',size=16)
            plt.ylim(-0.3,len(filterlist)-0.3)
    plt.tight_layout()
    plt.legend(loc='upper center', bbox_to_anchor=(-0.5, -0.15),
              fancybox=True, shadow=True, ncol=5, fontsize=16)
    plt.suptitle(r'Stellar H$\alpha$ brightness variability in 30-dor', y=1.03,fontsize=25)
    plt.savefig('output.png', dpi=200)
    plt.show()

In [ ]:
try:
    generate_plots(cleaned_df_pms, high_var=False)
except:
    cleaned_df_pms = prepare_df(phot_df_pms_stars)    
    generate_plots(cleaned_df_pms, high_var=False)

In [ ]:
cleaned_df_all = prepare_df(phot_df_redgiants)
generate_plots(cleaned_df_all)